In [ ]:
!pip install python-dotenv

In [ ]:
!pip install openai faiss-cpu numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 55.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# Get the API key from environment variables
openai_api_key = userdata.get("OPENAI_API_KEY")
import os
os.environ["OPENAI_API_KEY"] = openai_api_key


In [ ]:
from openai import OpenAI

client = OpenAI()



In [ ]:
from dataclasses import dataclass
from typing import Dict, List

@dataclass
class VehicleState:
    vehicle_id: str
    telemetry: Dict[str, float]
    anomaly: str | None = None
    retrieved_docs: List[str] = None
    diagnosis: str | None = None
    decision: str | None = None


In [ ]:
AUTOMOTIVE_KB = [
    "High engine temperature may indicate coolant leak or radiator failure.",
    "Low battery voltage often suggests battery degradation or alternator issues.",
    "Brake pressure loss is usually caused by hydraulic fluid leakage.",
    "Engine overheating can cause permanent engine damage if ignored.",
    "Brake system faults are safety critical and require immediate immobilization."
]


In [ ]:
import faiss
import numpy as np

def embed_texts(texts):
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=texts
    )
    return np.array([e.embedding for e in response.data]).astype("float32")

embeddings = embed_texts(AUTOMOTIVE_KB)
index = faiss.IndexFlatL2(len(embeddings[0]))
index.add(embeddings)


In [ ]:
class Agent:
    def run(self, state: VehicleState) -> VehicleState:
        raise NotImplementedError


In [ ]:
class TelemetryAgent(Agent):
    def run(self, state):
        print("[TelemetryAgent]")
        return state


In [ ]:
class AnomalyAgent(Agent):
    def run(self, state):
        print("[AnomalyAgent]")

        t = state.telemetry
        if t["engine_temp"] > 110:
            state.anomaly = "High engine temperature"
        elif t["battery_voltage"] < 11.5:
            state.anomaly = "Low battery voltage"
        elif t["brake_pressure"] < 20:
            state.anomaly = "Brake pressure loss"
        else:
            state.anomaly = "Normal"

        return state


In [ ]:
class RetrievalAgent(Agent):
    def run(self, state):
        print("[RetrievalAgent]")

        query_embedding = embed_texts([state.anomaly])
        _, indices = index.search(query_embedding, k=2)

        state.retrieved_docs = [AUTOMOTIVE_KB[i] for i in indices[0]]
        return state


In [ ]:
class LLMDiagnosisAgent(Agent):
    def run(self, state):
        print("[LLMDiagnosisAgent]")

        context = "\n".join(state.retrieved_docs)

        prompt = f"""
        You are an automotive diagnostics expert.

        Telemetry:
        {state.telemetry}

        Detected anomaly:
        {state.anomaly}

        Relevant technical knowledge:
        {context}

        Provide the most likely root cause.
        Be precise.
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )

        state.diagnosis = response.choices[0].message.content.strip()
        return state


In [ ]:
class LLMDecisionAgent(Agent):
    def run(self, state):
        print("[LLMDecisionAgent]")

        prompt = f"""
        You are a fleet safety decision AI.

        Diagnosis:
        {state.diagnosis}

        Decide the best action considering:
        - Human safety (highest priority)
        - Vehicle protection
        - Cost

        Respond with ONE clear action.
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )

        state.decision = response.choices[0].message.content.strip()
        return state


In [ ]:
class ReportAgent(Agent):
    def run(self, state):
        print("\n[ReportAgent] FINAL REPORT\n")

        print(f"""
Vehicle ID : {state.vehicle_id}
Telemetry  : {state.telemetry}
Anomaly    : {state.anomaly}

Retrieved Knowledge:
- {state.retrieved_docs[0]}
- {state.retrieved_docs[1]}

Diagnosis  : {state.diagnosis}
Decision   : {state.decision}
""")
        return state


In [ ]:
class Orchestrator:
    def __init__(self, agents):
        self.agents = agents

    def run(self, state):
        for agent in self.agents:
            state = agent.run(state)
        return state


In [ ]:
if __name__ == "__main__":
    vehicle = VehicleState(
        vehicle_id="CAR-2049",
        telemetry={
            "engine_temp": 125,
            "battery_voltage": 12.3,
            "brake_pressure": 32
        }
    )

    orchestrator = Orchestrator([
        TelemetryAgent(),
        AnomalyAgent(),
        RetrievalAgent(),
        LLMDiagnosisAgent(),
        LLMDecisionAgent(),
        ReportAgent()
    ])

    orchestrator.run(vehicle)


[TelemetryAgent]
[AnomalyAgent]
[RetrievalAgent]
[LLMDiagnosisAgent]
[LLMDecisionAgent]

[ReportAgent] FINAL REPORT


Vehicle ID : CAR-2049
Telemetry  : {'engine_temp': 125, 'battery_voltage': 12.3, 'brake_pressure': 32}
Anomaly    : High engine temperature

Retrieved Knowledge:
- High engine temperature may indicate coolant leak or radiator failure.
- Engine overheating can cause permanent engine damage if ignored.

Diagnosis  : Based on the provided telemetry and anomaly detection:

The engine temperature of 125 degrees is unusually high, which suggests that there might be a cooling system issue. The most likely root cause for this high engine temperature is a coolant leak or radiator failure. A coolant leak can reduce the coolant level below the necessary threshold, preventing effective heat dissipation from the engine. Alternatively, a radiator failure, which can involve a blocked radiator or a malfunctioning radiator fan, will also result in inadequate cooling of the engine. Both 